In [15]:
import pandas as pd
from g4f.client import Client
import re

In [16]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    加載 CSV 文件並處理錯誤行。
    """
    try:
        return pd.read_csv(file_path, on_bad_lines="skip", encoding="utf-8")
    except pd.errors.ParserError as e:
        raise Exception(f"CSV 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")



In [17]:
def generate_summary(data: pd.DataFrame):
    """
    生成數據描述性統計。
    Args:
        data (pd.DataFrame): 輸入數據框。
    Returns:
        pd.DataFrame: 描述性統計。
    """
    try:
        return data.describe()
    except Exception as e:
        raise Exception(f"生成描述性統計時發生錯誤：{e}")

In [18]:
def construct_prompt(user_input: str, summary_stats: pd.DataFrame) -> tuple:
    """
    構建系統與用戶提示。
    Args:
        user_input (str): 用戶查詢。
        summary_stats (pd.DataFrame): 描述性統計。
    Returns:
        tuple: (系統提示, 用戶提示)
    """
    system_prompt = (
        "你是一位專業的數據科學家，擅長分析數據並從中提取見解。"
        "以下是一些統計數據，請根據這些數據提供深入分析和建議：\n"
        "1. 描述數據的整體特性，例如平均值、標準差、數據分佈等。\n"
        "2. 尋找可能的異常值或數據問題，並評估其對分析結果的影響。\n"
        "3. 如果有類別型數據，請分析其分佈特性和可能的相關性。\n"
        "4. 根據數據，提供有助於業務決策的具體建議，例如潛在的改進方向或應關注的特定方向。\n"
        "5. 將您的分析結論簡潔清楚地表達，並建議下一步的數據收集或分析方向。"
    )
    user_prompt = (
        f"以下是數據的描述性統計結果：\n{summary_stats.to_string()}\n"
        f"請根據這些數據回答以下問題：{user_input}"
    )
    return system_prompt, user_prompt


In [19]:
def generate_report(file_path: str, user_input: str) -> str:
    """
    基於自然語言查詢生成分析報告。
    Args:
        file_path (str): 數據文件路徑。
        user_input (str): 用戶的自然語言查詢。
    Returns:
        str: 分析報告或錯誤信息。
    """
    try:
        # 1. 加載數據
        data = load_data(file_path)

        # 2. 生成描述性統計
        summary_stats = generate_summary(data)

        # 3. 構建 GPT 模型請求的提示
        system_prompt, user_prompt = construct_prompt(user_input, summary_stats)

        # 4. 初始化 GPT 客戶端並生成報告
        client = Client()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"生成報告時發生錯誤：{e}"



In [1]:
if __name__ == "__main__":
    # 示例使用
    file_path = file_path = r".\csv.ingredient\tsmc_operating_profit_margin_2010_2023.csv"

    user_query = "請分析數據並提供詳細的見解和建議，並提供報告。"  # 自然語言查詢

    try:
        report = generate_report(file_path, user_query)
        print("生成的報告：")
        print(report)
    except Exception as e:
        print(f"程序運行失敗：{e}")

程序運行失敗：name 'generate_report' is not defined
